In [ ]:
import sys 
sys.path.append('/opt/deep_Learning/utilities")
from spark_setup import SetupEnvironment 
environment = SetupEnvironment(
                conda_env = "my_env", # should match with jupyter kernel used (e.g. my_env or deep_learning)
                project_name = "NoLivyExampleProject", # defaults to Domino project name if not set
                additional_settings = {

                    #"spark.executor_memory": "130G" ,
                    "spark.driver.memory": "130G" ,
                    #"spark.executor.cores": "2",
                    #cores max == 24 instances * 20 each(large) * 0.8 = rough384
                    #"spark.cores.max": "180",
                    #"spark.driver.memoryOverhead": "20G", 
                    #"spark.executor_memoryOverhead" : "20G" ,
                    #"spark.sql.autoBroadcastJoinThreshold" : '-1', 
                    #"spark.sql.shuffle.partitions"  : "200",
                    #"spark.shuffle.service.enabled" : 'true',
                    #'spark.excludeOnFailure.enabled' : "true",
                    #"spark.excludeOnFailule.application.fetchFailure.enabled":'true' ,
                    #"spark.executorEnv_PYARROW_IGNORE_TIMEZONE":"1" ,
                    #"spark.sql.execution.arrow.pyspark.enabled" : 'true',
                    #"spark.sql.execution.arrow.pyspark.fallback.enabled":'true',
                } # additional spark configuration (optional)
)

spark = environment.setup_spark()
dcRead = environment.setup_DataCatalog()
s3 = environment.setup_s3()

#'spark.files.fetchTimeout' : '300s' 
#'spark.network.timeout' : '360s'

### Import Functions

In [ ]:
import s3fs 
import pickle 
import pyarrow 
import os 
import sys
import pandas as pd 
import datetime 
import numpy as np
from pyarrow import parquet as pa 
import json
import numpy as np 
import pandas as pd 
from tensorflow import keras 
from tensorflow. keras import layers 
from matplotlib import pyplot as plt 
import pyspark.sql.functions as F
pd.set_option('display.max_rows', None)

In [ ]:
#vizualization modules

from plotly.offline import download_plotlyis, plot, iplot 
import plotly.graph_objs as go 
import plotly.io as pio 
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### Define Functions

In [ ]:
def read_pipeline_parquet(s3a_bucket_str, dataframe_name, date_start, date_end=None, verbose=False, version=None):

    #writes loaded data into parquet for future consumption 
    
    if date_start == None:
        file_path = s3a_bucket_str+dataframe_name 
    elif date_end != None:
        file_path = s3a_bucket_str+dataframe_name+'_{}'.format(date_start)+'_{}'.format(date_end)
    else:
        file_path = s3a_bucket_str+dataframe_name+'_{}'.format(date_start)

    if verbose==True:
        print(f'Reading from {file_patht}')
    else:
        pass
    if version == None:
        pass
    else:
        file_path =file_path +'_v'+str(version)
    new_file = spark.read.parquet(file_path)
    return new_file

### Define Variables

In [ ]:
#For Reading/Writing
PROBE_DATE = '2022-01-20'
CURRENT_VERSION = 0
BUCKET_STRING = "s3a://9262-4618-2427-mrp-cat-kms/users/k30577/"
RANDOM_SEED = 42
TRAIN_PROPORTION = 0.75
#SLICES = 6 #(chosen in last step of process)

### Read in Sampled IV Chain Data from 2.5

Run Code from one of next two blocks, Sampling Method 1 contains exactly 1 subwindow from each chain, whereas Sampling Method 2 contains a 0.5% sample of all subwindows. Each is similar in overall size. SM1 was used for retrains 1,2. Where as SM2 was used for retrain 3, Sampling method 1



In [ ]:
#Sampling Method 1

Final_IV_Data_Sample = read_pipeline_parquet(BUCKET_STRING, "actrain9_IV_normalized_rows_030_1ea",PROBE_DATE, version = CURRENT_VERSION)

In [ ]:
Final_IV_Data_Sample

In [ ]:
#Sampling Method 2

Final_IV_Data_Sample = read_pipeline_parquet(BUCKET_STRING,"actrain9_IV_normalized_rows_005", PROBE_DATE, version=CURRENT_VERSION)

In [ ]:
Final_IV_Data_Sample

In [ ]:
# change data to pandas format
Final_IV_Data_Sample_Pre_PD = Final_IV_Data_Sample.select('chain_id', 'all_items')


In [ ]:
Final_IV_Data_Sample_PD = Final_IV_Data_Sample_Pre_PD.toPandas()

In [ ]:
Final_IV_Data_Sample_PD.describe()

In [ ]:
pd.DataFrame(Final_IV_Data_Sample_PD.isna().sum()[Final_IV_Data_Sample_PD.isna().sum() > 0], columns = ['count nan values'])

In [ ]:
Final_IV_Data_Sample_PD.info()

#### Get Rid of NaN rows and split and into training and validation (no test for unsupervised data) (Split By Chain)


In [ ]:
Final_IV_Data_Sample_PD['np_items'] = Final_IV_Data_Sample_PD['all_items'J.to_numpy()
Final_IV_Data_Sample_PD['np_items_1st'] = Final_IV_Data_Sample_PD['np_items'].str[0]
Final_IV_Data_Sample_PD_DeNan= Final_IV_Data_Sample_PD[Final_IV_Data_Sample_PD['np_items_1st'].isna()==False]

In [ ]:
Final_IV_Data_Sample_PD_DeNan.describe()

In [ ]:
pd.DataFrame(Final_IV_Data_Sample_PD_DeNan.isna().sum()[Final_IV_Data_Sample_PD_DeNan.isna().sum() > 0], columns = ['count nan values'])

In [ ]:
Final_IV_Data_Sample_PD_DeNan.info()

### Split into Training and Validation Set

In [ ]:
num_events = len(Final_IV_Data_Sample_PD_DeNan)

In [ ]:
#num_events = Len(Final_IV_Data_002_Sample_PD_DeNan)
train_val_df = Final_IV_Data_Sample_PD_DeNan.sample(frac=1, random_state = RANDOM_SEED).reset_index()
train_df = train_val_df.loc[: int(TRAIN_PROPOTION*num_events)]
val_df = train_val_df.loc[int(TRAIN_PROPOTION*num_events)+1:]

### Create and Reshape Training and Validation Sets

In [ ]:
X_train = np.stack(train_df['all_items'].to_numpy(), axis=0)
X_train = X_train.reshape(*(list(X_train.shape)+[1]))

In [ ]:
X_train

In [ ]:
X_val = np.stack(val_df['all_items'].to_numpy(), axis=0)
X_val = X_val.reshape(*(list(X_val.shape)+[1]))

In [ ]:
X_val

### Build Model Architecture

Modeling Decisions:

The original autoencoder artifact had 64 neurons, informing the NUM_NEURONS decision.

The loss function was changed from 'mae' to 'mse' between retraining efforts, the latter decision brought the mae scores for the validatio

line with the scores given by the artifact.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

DROPOUT = 0.2
NUM_NEURONS = 64 

model = Sequential()
model.add(LSTM(NUM_NEURONS, input_shape=(X_train.shape[1], X_train.shape[2])))
modeL.add(Dropout(rate=DROPOUT))
model.add(RepeatVector(X_train.shape[1]))
model.add(LSTM(NUM_NEURONS,return_sequences=True)) 
model.add(Dropout(rate=DROPOUT)) 
model.add(TimeDistributed(Dense(X_train.shape[2]))) 
model.compile(optimizer='adam', loss='mse')
model.summary()

### Train Model

In [ ]:
model_ts = datetime.datetime.now()
EPOCHS = 10
BATCH_SIZE = 2048*2

history = model.fit(

    X_train, 
    X_train, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    validation_data=(X_val,X_val), 
    callbacks=[
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, mode ="min")
            ]
)


### validation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.style.use('dark_background')
import warnings
warnings.filterwarnings('ignore')


In [ ]:
plt.plot(history.history['loss'], c = 'b', label='train')
plt.plot(history.history['val_loss'], c = 'y', label='validation')
plt.legend()
plt.title("Reconstruction Loss ")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.show()

In [ ]:
x_train_pred = model.predict(X_train)
x_val_pred = model.predict(X_val)

In [ ]:
mets = []
train = []
val = []

mets.append('mean_squared_error')
train.append(mean_squared_error(x_train, x_train_pred))
val.append(mean_squared_error(x_val, x_val_pred))

mets.append('mean_absolute_error')
train.append(mean_absolute_error(x_train, x_train_pred))
val.append(mean_absolute_error(x_val, x_val_pred))

mets.append('r2_multi_output_regres')
train.append(MultiOutputRegressor(Ridge(random_state=123)).fit(x_train, x_train_pred).score(x_train, x_train_pred))
val.append(MultiOutputRegressor(Ridge(random_state=123)).fit(x_val, x_val_pred).score(x_val, x_val_pred))

mets.append('r2_sklearn')
train.append(r2_score(x_train, x_train_pred))
val.append(r2_score(x_val, x_val_pred))

metrics = pd.DataFrame({'metrics' : mets, 'train_set' : train, 'validation_set' : val })
metrics

In [ ]:
data = [x_train.tolist(), x_train_pred.tolist()]
chi_square_dict = {'stats':chi2_contingency(data)[0], 'p':chi2_contingency(data)[1],'dof':chi2_contingency(data)[2]}
print(f'train_chi2_contingency: \n {chi_square_dict}')

In [ ]:
data = [x_val.tolist(), x_val_pred.tolist()]
chi_square_dict = {'stats':chi2_contingency(data)[0], 'p':chi2_contingency(data)[1],'dof':chi2_contingency(data)[2]}
print(f'validation_chi2_contingency: \n {chi_square_dict}')

### Save Model

autoencoder_retrain_0 - 256 Neurons, MAE, Sampling Method 1 (Not placed in final artifacts folder considered in model selection, different number of parameters.)

autoencoder_retrain_1 - 64 Neurons, MAE, Sampling Method 1

*autoencoder_retrain_2 - 64 Neurons, MSE, Sampling Method 1 (Current Best Model)

autoencoder_retrain_3 - 64 Neurons, MSE, Sampling Method 2

model_mse. save("autoencoder_retrain_3")

In [ ]:
model.save("autoencoder_retrain_3")